In [38]:
import os
import json
import pandas as pd
import traceback

In [39]:
from langchain_openai import ChatOpenAI

In [40]:
from dotenv import load_dotenv


load_dotenv()  # take environment variables

True

In [41]:
KEY=os.getenv("OPENAI_API_KEY")

In [42]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [43]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x13322e700>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x13322ee20>, root_client=<openai.OpenAI object at 0x13322e820>, root_async_client=<openai.AsyncOpenAI object at 0x13322eb80>, temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [44]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [45]:
RESPONSE_JSON={
    "1": {
        "no": "1",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    },
    
    "2": {
        "no": "2",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    },
    
    "3": {
        "no": "3",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    }
    
    
}

In [46]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""


In [47]:
quiz_generation_prompt = PromptTemplate(
   input_variables=["text","number","subject","tone","response_json"],
   template=TEMPLATE
)

In [48]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [49]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of teh question and give a complete analysis of the quiz if the students
will be able to unserstand the questions and answer them. Only use at max 50 words for complexity analysis.
if the quiz is not at par with the cognitive and analytical abilities of the students,\
update tech quiz questions which needs to be changed  and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}


Check from an expert English Writer of the above quiz:
"""


In [50]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [51]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [52]:
# This is an Overall Chain where we run the two chains in Sequence
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                       output_variables=["quiz", "review"], verbose=True,)

In [53]:
file_path="/Users/pavannagabathula/mcqgen/data.txt"

In [54]:
file_path

'/Users/pavannagabathula/mcqgen/data.txt'

In [55]:
with open(file_path,'r') as file:
   TEXT =file.read()


# %%
print("File content length:", len(TEXT))


File content length: 3349


In [56]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contri

In [57]:
#serialize the python dictionary into a JSON- formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [58]:
NUMBER=5
SUBJECT="machine learning"
TONE='simple'

In [59]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking


#How to setup Token Usage tracking in langchain


with get_openai_callback() as cb:
   response=generate_evaluate_chain(
   {
       "text":TEXT,
       "number":NUMBER,
       "subject":SUBJECT,
       "tone":TONE,
       "response_json": json.dumps(RESPONSE_JSON)
   }
   )


/var/folders/77/nwcv_cts2md93pltv_dhpd140000gn/T/ipykernel_66050/4057235323.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons 

In [60]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2715
Prompt Tokens:1828
Completion Tokens:887
Total Cost:0.0022445


In [61]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nThe earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive s

In [62]:
quiz=response.get("quiz")

In [63]:
json.loads(quiz)

{'1': {'no': '1',
  'mcq': "When was the term 'machine learning' coined?",
  'options': {'a': '1949', 'b': '1959', 'c': '1969', 'd': '1979'},
  'correct': 'b'},
 '2': {'no': '2',
  'mcq': "Who introduced the theoretical neural structure in the book 'The Organization of Behavior'?",
  'options': {'a': 'Tom M. Mitchell',
   'b': 'Walter Pitts',
   'c': 'Donald Hebb',
   'd': 'Warren McCulloch'},
  'correct': 'c'},
 '3': {'no': '3',
  'mcq': "What was the purpose of the experimental 'learning machine' Cybertron developed by Raytheon Company?",
  'options': {'a': 'To analyse weather patterns',
   'b': 'To analyse sonar signals, electrocardiograms, and speech patterns',
   'c': 'To play chess',
   'd': 'To send emails'},
  'correct': 'b'},
 '4': {'no': '4',
  'mcq': 'According to Tom M. Mitchell, what does a computer program do in machine learning with respect to tasks, experience, and performance?',
  'options': {'a': 'It learns from experience E with respect to some class of tasks T and p

In [64]:
quiz=json.loads(quiz)

In [65]:
quiz_table_data = []
for key, value in quiz.items():
   mcq = value["mcq"]
   options = " | ".join(


       [f"{option}: {option_value}"
        for option, option_value in value["options"].items()
        ]
   )
   correct = value["correct"]
   quiz_table_data.append({"MCQ": mcq,"Choices": options,"Correct": correct})

In [66]:
quiz_table_data

[{'MCQ': "When was the term 'machine learning' coined?",
  'Choices': 'a: 1949 | b: 1959 | c: 1969 | d: 1979',
  'Correct': 'b'},
 {'MCQ': "Who introduced the theoretical neural structure in the book 'The Organization of Behavior'?",
  'Choices': 'a: Tom M. Mitchell | b: Walter Pitts | c: Donald Hebb | d: Warren McCulloch',
  'Correct': 'c'},
 {'MCQ': "What was the purpose of the experimental 'learning machine' Cybertron developed by Raytheon Company?",
  'Choices': 'a: To analyse weather patterns | b: To analyse sonar signals, electrocardiograms, and speech patterns | c: To play chess | d: To send emails',
  'Correct': 'b'},
 {'MCQ': 'According to Tom M. Mitchell, what does a computer program do in machine learning with respect to tasks, experience, and performance?',
  'Choices': 'a: It learns from experience E with respect to some class of tasks T and performance measure P | b: It learns from experience E with respect to some class of tasks T and performance measure Q | c: It learns

In [67]:
pd.DataFrame (quiz_table_data)

,MCQ,Choices,Correct
0,When was the term 'machine learning' coined?,a: 1949 | b: 1959 | c: 1969 | d: 1979,b
1,Who introduced the theoretical neural structur...,a: Tom M. Mitchell | b: Walter Pitts | c: Dona...,c
2,What was the purpose of the experimental 'lear...,a: To analyse weather patterns | b: To analyse...,b
3,"According to Tom M. Mitchell, what does a comp...",a: It learns from experience E with respect to...,a
4,What are the two objectives of modern-day mach...,a: To classify data based on models developed ...,b


In [68]:
quiz=pd.DataFrame (quiz_table_data)

In [70]:
quiz.to_csv("machinelearning.csv",index=False)


In [3]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'07_22_2025_14_58_57'